##### TODO Recording

- Run the first few cells, then switch over to start the source (will be in recording notes below)

In [0]:
import json
from pyspark.sql.types import *
from pyspark.sql.functions import *
from datetime import datetime as dt

In [0]:
# # For 2.3.15 version and above, the configuration dictionary requires that connection string be encrypted.

In [0]:
primaryKey = "Endpoint=sb://loony-eventhub-namespace.servicebus.windows.net/;SharedAccessKeyName=RootManageSharedAccessKey;SharedAccessKey=eKnj1tqnC6UJdhA9I2IVN1rvh2oVcc6Ubd2Cnvr4PQ0="
entityPath = "EntityPath=loony-sales"

connectionString = primaryKey + ";" + entityPath

ehConf = {}

startingEventPosition = {
  "offset": "-1",  
  "seqNo": -1,            
  "enqueuedTime": None,  
  "isInclusive": True
}

ehConf['eventhubs.connectionString'] = sc._jvm.org.apache.spark.eventhubs.EventHubsUtils.encrypt(connectionString)
ehConf["eventhubs.startingPosition"] = json.dumps(startingEventPosition)

In [0]:
from pprint import pprint

pprint(ehConf)

{'eventhubs.connectionString': 'P4w5ooxK264fEn4fSjKMh0CgCpv6wQLbtLTOzbqmvtyM/MK1uyEyPEhrqFUohdaoEq1ipYfu2zWSoFKQ6knyBLS/5gUDr3ADZyLcTE6KvSMk7p/pKjtZca2Dq+6rahexk0vGX03dVL6MMXNSFPbwoq890m5/e3/SJ67TeuugVvq2PEnOvg2JIssGDiJmDM+V2WTcgrC+wMZv9qyZjhhsR5bJ26rx9e/YBFu5uAaZhHJ/Vo6oEqT3K+7E7Yk+ptFZ81e2MAzIP6G58yc52tJ70A==',
 'eventhubs.startingPosition': '{"offset": "-1", "seqNo": -1, "enqueuedTime": '
 'null, "isInclusive": true}'}

##### TODO Recording

- First we are adding some data to event hub
- Go to SourceData and run the code
- This is Batch #1 from superstore.txt

        event_data_batch.add(EventData(
            '{"State":"Kentucky","Category":"Furniture","Sub-Category":"Bookcases","Sales":261.96,"Quantity":2,"Profit":41.9136, "Timestamp": "2021-10-09 07:22:13"}'))      
        event_data_batch.add(EventData(
            '{"State":"Kentucky","Category":"Furniture","Sub-Category":"Chairs","Sales":731.94,"Quantity":3,"Profit":219.582, "Timestamp": "2021-10-09 07:23:34"}'))
        event_data_batch.add(EventData(
            '{"State":"California","Category":"Office Supplies","Sub-Category":"Labels","Sales":14.62,"Quantity":2,"Profit":6.8714, "Timestamp": "2021-10-09 07:22:45"}'))
        event_data_batch.add(EventData(
            '{"State":"Florida","Category":"Furniture","Sub-Category":"Tables","Sales":957.5775,"Quantity":5,"Profit":-383.031, "Timestamp": "2021-10-09 07:23:03"}'))
        event_data_batch.add(EventData(
            '{"State":"Florida","Category":"Office Supplies","Sub-Category":"Storage","Sales":22.368,"Quantity":2,"Profit":2.5164, "Timestamp": "2021-10-09 07:25:55"}'))
        event_data_batch.add(EventData(
            '{"State":"California","Category":"Furniture","Sub-Category":"Furnishings","Sales":48.86,"Quantity":7,"Profit":14.1694, "Timestamp": "2021-10-09 07:24:37"}'))

- Expand the dataframe read and show the structure

In [0]:
superstore_df = spark.readStream \
                     .format("eventhubs") \
                     .options(**ehConf) \
                     .load()

In [0]:
schema = StructType([StructField("State", StringType(), True),
                     StructField("Category", StringType(), True),
                     StructField("Sub-Category", StringType(), True),
                     StructField("Sales", FloatType(), True),
                     StructField("Quantity", IntegerType(), True),
                     StructField("Profit", FloatType(), True),
                     StructField("Timestamp", TimestampType(), True)
                    ])

In [0]:
superstore_data = superstore_df.selectExpr("cast(Body as string) as json")\
                               .select(from_json("json", schema) \
                               .alias("data")) \
                               .select("data.*")
display(superstore_data)

State,Category,Sub-Category,Sales,Quantity,Profit,Timestamp
Kentucky,Furniture,Bookcases,261.96,2,41.9136,2021-10-09T07:22:13.000+0000
Kentucky,Furniture,Chairs,731.94,3,219.582,2021-10-09T07:23:34.000+0000
California,Office Supplies,Labels,14.62,2,6.8714,2021-10-09T07:22:45.000+0000
Florida,Furniture,Tables,957.5775,5,-383.031,2021-10-09T07:23:03.000+0000
Florida,Office Supplies,Storage,22.368,2,2.5164,2021-10-09T07:25:55.000+0000
California,Furniture,Furnishings,48.86,7,14.1694,2021-10-09T07:24:37.000+0000
California,Office Supplies,Art,7.28,4,1.9656,2021-10-09T07:30:09.000+0000
California,Technology,Phones,907.152,6,90.7152,2021-10-09T07:24:49.000+0000
California,Office Supplies,Binders,18.504,3,5.7825,2021-10-09T07:38:26.000+0000
Kentucky,Office Supplies,Appliances,114.9,5,34.47,2021-10-09T07:39:01.000+0000


##### Watermark with TumblingWindow

In [0]:
# Here we will display the data with windowing
# Not with watermark

In [0]:
windowed_count = superstore_data.groupBy(window(superstore_data.Timestamp, "2 minutes")) \
                                .count()
                              
display(windowed_count)

window,count
"List(2021-10-09T07:30:00.000+0000, 2021-10-09T07:32:00.000+0000)",1
"List(2021-10-09T07:38:00.000+0000, 2021-10-09T07:40:00.000+0000)",2
"List(2021-10-09T07:24:00.000+0000, 2021-10-09T07:26:00.000+0000)",3
"List(2021-10-09T07:22:00.000+0000, 2021-10-09T07:24:00.000+0000)",4


In [0]:
windowed_count.writeStream\
              .queryName("windowed_count")\
              .outputMode("update")\
              .format("memory")\
              .start()

Out[26]: <pyspark.sql.streaming.StreamingQuery at 0x7fac08e5cfa0>

###### TODO Recording

- Run the cell below only after the query in the previous cell is complete otherwise you will not get results

In [0]:
%sql

select * from windowed_count

window,count
"List(2021-10-09T07:24:00.000+0000, 2021-10-09T07:26:00.000+0000)",2
"List(2021-10-09T07:22:00.000+0000, 2021-10-09T07:24:00.000+0000)",4
"List(2021-10-09T07:30:00.000+0000, 2021-10-09T07:32:00.000+0000)",1
"List(2021-10-09T07:24:00.000+0000, 2021-10-09T07:26:00.000+0000)",3


In [0]:
windowed_count_withwatermark = superstore_data.withWatermark("Timestamp", "3 minutes") \
                                              .groupBy(window(superstore_data.Timestamp, "2 minutes")) \
                                              .count()

display(windowed_count_withwatermark)

window,count
"List(2021-10-09T07:30:00.000+0000, 2021-10-09T07:32:00.000+0000)",1
"List(2021-10-09T07:38:00.000+0000, 2021-10-09T07:40:00.000+0000)",2
"List(2021-10-09T07:24:00.000+0000, 2021-10-09T07:26:00.000+0000)",3
"List(2021-10-09T07:22:00.000+0000, 2021-10-09T07:24:00.000+0000)",4


In [0]:
windowed_count_withwatermark.writeStream\
              .queryName("windowed_count_withwatermark")\
              .outputMode("update")\
              .format("memory")\
              .start()

Out[28]: <pyspark.sql.streaming.StreamingQuery at 0x7fac08e5c8e0>

###### TODO Recording

- Run the cell below only after the query in the previous cell is complete otherwise you will not get results

In [0]:
%sql

select * from windowed_count_withwatermark

window,count
"List(2021-10-09T07:24:00.000+0000, 2021-10-09T07:26:00.000+0000)",2
"List(2021-10-09T07:22:00.000+0000, 2021-10-09T07:24:00.000+0000)",4
"List(2021-10-09T07:30:00.000+0000, 2021-10-09T07:32:00.000+0000)",1
"List(2021-10-09T07:24:00.000+0000, 2021-10-09T07:26:00.000+0000)",3


#### TODO Recording

- Now let's add some more data

        event_data_batch.add(EventData(
            '{"State":"California","Category":"Office Supplies","Sub-Category":"Art","Sales":7.28,"Quantity":4,"Profit":1.9656,"Timestamp": "2021-10-09 07:30:09"}'))
        event_data_batch.add(EventData(
            '{"State":"California","Category":"Technology","Sub-Category":"Phones","Sales":907.152,"Quantity":6,"Profit":90.7152,"Timestamp": "2021-10-09 07:24:49"}'))
            
- There should be no difference in the output, Spark *may* drop data which comes beyond the watermark
- Please explicitly run both cells which have the SQL query

In [0]:
avg_sales = superstore_data.withWatermark("Timestamp", "30 seconds") \
                           .groupBy(window(superstore_data.Timestamp, "1 minutes")) \
                           .agg({"Sales": "avg"}) \
                           .orderBy("window.start")
                              

display(avg_sales)

window,avg(Sales)
"List(2021-10-09T07:22:00.000+0000, 2021-10-09T07:23:00.000+0000)",138.2899956703186
"List(2021-10-09T07:23:00.000+0000, 2021-10-09T07:24:00.000+0000)",844.7587585449219
"List(2021-10-09T07:24:00.000+0000, 2021-10-09T07:25:00.000+0000)",478.00598907470703
"List(2021-10-09T07:25:00.000+0000, 2021-10-09T07:26:00.000+0000)",22.368000030517578
"List(2021-10-09T07:30:00.000+0000, 2021-10-09T07:31:00.000+0000)",7.28000020980835
"List(2021-10-09T07:38:00.000+0000, 2021-10-09T07:39:00.000+0000)",18.503999710083008
"List(2021-10-09T07:39:00.000+0000, 2021-10-09T07:40:00.000+0000)",114.9000015258789


In [0]:
avg_profit_state = superstore_data.withWatermark("Timestamp", "30 seconds") \
                                  .groupBy("State", window(superstore_data.Timestamp, "3 minutes")) \
                                  .agg({"Profit": "avg"}) \
                                  .orderBy("window.start")
                              

display(avg_profit_state)

State,window,avg(Profit)
Florida,"List(2021-10-09T07:21:00.000+0000, 2021-10-09T07:24:00.000+0000)",-383.031005859375
California,"List(2021-10-09T07:21:00.000+0000, 2021-10-09T07:24:00.000+0000)",6.871399879455566
Kentucky,"List(2021-10-09T07:21:00.000+0000, 2021-10-09T07:24:00.000+0000)",130.74780082702637
California,"List(2021-10-09T07:24:00.000+0000, 2021-10-09T07:27:00.000+0000)",52.44230127334595
Florida,"List(2021-10-09T07:24:00.000+0000, 2021-10-09T07:27:00.000+0000)",2.516400098800659
California,"List(2021-10-09T07:30:00.000+0000, 2021-10-09T07:33:00.000+0000)",1.9656000137329102
California,"List(2021-10-09T07:36:00.000+0000, 2021-10-09T07:39:00.000+0000)",5.78249979019165
Kentucky,"List(2021-10-09T07:39:00.000+0000, 2021-10-09T07:42:00.000+0000)",34.470001220703125


##### Watermark With SlidingWindow

In [0]:
avg_sales_category = superstore_data.withWatermark("Timestamp", "1 hour") \
                                    .groupBy("Category", window(superstore_data.Timestamp, "3 minutes", "1 minute")) \
                                    .agg({"Sales": "avg"}) \
                                    .orderBy("window.start")
                              

display(avg_sales_category)

Category,window,avg(Sales)
Office Supplies,"List(2021-10-09T07:20:00.000+0000, 2021-10-09T07:23:00.000+0000)",14.619999885559082
Furniture,"List(2021-10-09T07:20:00.000+0000, 2021-10-09T07:23:00.000+0000)",261.9599914550781
Furniture,"List(2021-10-09T07:21:00.000+0000, 2021-10-09T07:24:00.000+0000)",650.4925028483073
Office Supplies,"List(2021-10-09T07:21:00.000+0000, 2021-10-09T07:24:00.000+0000)",14.619999885559082
Furniture,"List(2021-10-09T07:22:00.000+0000, 2021-10-09T07:25:00.000+0000)",500.08437728881836
Technology,"List(2021-10-09T07:22:00.000+0000, 2021-10-09T07:25:00.000+0000)",907.1519775390625
Office Supplies,"List(2021-10-09T07:22:00.000+0000, 2021-10-09T07:25:00.000+0000)",14.619999885559082
Office Supplies,"List(2021-10-09T07:23:00.000+0000, 2021-10-09T07:26:00.000+0000)",22.368000030517578
Technology,"List(2021-10-09T07:23:00.000+0000, 2021-10-09T07:26:00.000+0000)",907.1519775390625
Furniture,"List(2021-10-09T07:23:00.000+0000, 2021-10-09T07:26:00.000+0000)",579.4591725667318


##### TODO Recording

- Add some more data and show the output of the sliding window (only the previous query)

        event_data_batch.add(EventData(
            '{"State":"California","Category":"Office Supplies","Sub-Category":"Binders","Sales":18.504,"Quantity":3,"Profit":5.7825,"Timestamp": "2021-10-09 07:38:26"}'))
        event_data_batch.add(EventData(
            '{"State":"Kentucky","Category":"Office Supplies","Sub-Category":"Appliances","Sales":114.9,"Quantity":5,"Profit":34.47,"Timestamp": "2021-10-09 07:39:01"}'))

- Scroll to the bottom and show rows